In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

모델 구조

Meta Data 생성  
불균형 데이터 조정  
결측치 처리  
Feature Engineering  
범주형 변수 인코딩  
Mean Encoding  
One-hot Encoding  
PolynomialFeatures  
Feature Selection  
VarianceThreshold  
SelectFromModel  
Feature Scaling  
Modeling  

<대회 설명>  

Porto Seguro는 브라질의 자동차 보험 회사.    
보험 청구할 확률이 높은 차주한테 보험료를 높게 청구하고, 아닌 차주는 낮게 보험료를 청구     

<대회 목적>

어떤 차주가 내년에 보험 청구 할 확률 예측

<데이터 특징>

테스트 데이터 > 훈련 데이터
결측치의 값이 -1로 주어짐
실제 기업 데이터이기 때문에 feature를 비식별화 해놨음 (그렇기 때문에 난이도가 높은 쪽에 속함),  
지니 예상으로는 사고 횟수, 보험같이든 가족 수 등등이 아닐까?         
target = 0 : 보험 청구 no / target = 1 : 보험 청구 yes  
<데이터 평가>

predict_proba라는 함수를 사용해서 값을 예측 (확률 값)  

대회의 평가지표는 특이하게 Normalized Gini Coefficient를 사용함  
지니 계수 : 경제적 불평등을 계수화할 때 주로 사용하는 지표
Normalized Gini Coefficient를 사용하는 이유:  
Imbalanced Class를 평가를 위한 임계값을 어떻게 정하느냐에 따라 예측값이 바뀜  
= ROC 커브로 확인 (면적으로 스코어를 매김) >> 비슷한 목적으로 Gini Coefficient가 쓰임  
= gini = 2 * AUC - 1

0~0.5의 값을 가짐
0.5에 가까울수록 좋은 분석임.



In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# from sklearn.preprocessing import Imputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.max_columns', 100)

In [3]:
train = pd.read_csv('../input/porto-seguros-safe-driver-prediction-train-data/train.csv')
test = pd.read_csv('../input/porto-seguros-safe-driver-prediction-test-data/test.csv')
# sub = pd.read_csv("/kaggle/input/bike-sharing-demand/sampleSubmission.csv")                   


In [4]:
train.head()

In [5]:
train.tail()

In [6]:
train.shape

In [7]:
train.drop_duplicates()
train.shape

No duplicate rows, so that's fine.

In [8]:
test.shape

In [9]:
test.drop_duplicates()
test.shape

In [10]:
train.info()

# Meta Data
role: input, ID, target  
level: nominal, interval, ordinal, binary  
keep: True or False  
dtype: int, float, str  

In [11]:
data = []
for f in train.columns:
    if f =='target':
        role = 'target'
    elif f == 'id':
        role = 'id'
    else:
        role = 'input'
   
    # if there is the bin string in the f or 'target' column, level is binary    
    if 'bin' in f or f == 'target':
        level = 'binary'
    elif 'cat' in f or f == 'id':
        level = 'nominal'
    # if trian's columns' dtypes are float, level is float.  
    elif train[f].dtype == float:
        level = 'float'
    elif train[f].dtype == int:
        level = 'ordinal'
        
   # Initialize keep to True for all variables except for id
    keep = True
    if f == 'id':
        keep = False
    

    dtype = train[f].dtype
    
    # DataFrame으로 만들기 위해 리스트에 append하기 전에 딕셔너리 타입으로 만들어주었음
    f_dict={
        'varname':f,
        'roel':role,
        'level':level,
        'keep':keep,
        'dtype': dtype
    }
    
    data.append(f_dict)
    
# 변수의 이름을 인덱스로 하는 데이터프레임을 만들어줌    
meta = pd.DataFrame(data, columns = ['varname', 'role', 'level', ' keep', ' dtype'])
meta.set_index('varname', inplace = True)

In [12]:
data = []
for f in train.columns:
    # Defining the role
    if f == 'target':
        role = 'target'
    elif f == 'id':
        role = 'id'
    else:
        role = 'input'
         
    # Defining the level
    if 'bin' in f or f == 'target':
        level = 'binary'
    elif 'cat' in f or f == 'id':
        level = 'nominal'
    elif train[f].dtype == float:
        level = 'interval'
    elif train[f].dtype == int:
        level = 'ordinal'
        
    # Initialize keep to True for all variables except for id
    keep = True
    if f == 'id':
        keep = False
    
    # trian set의 컬럼별 원래 dtype을 적어라. 
    dtype = train[f].dtype
    
    # Creating a Dict that contains all the metadata for the variable
    f_dict = {
        'varname': f,   # f에 들어가 있는 모든 train의 colums들 
        'role': role,
        'level': level,
        'keep': keep,
        'dtype': dtype
    }
    data.append(f_dict)
    
meta = pd.DataFrame(data, columns=['varname', 'role', 'level', 'keep', 'dtype'])
meta.set_index('varname', inplace=True)

In [13]:
meta

Example to extract all nominal variables that are not dropped

In [14]:
meta[(meta.level == 'nominal') & (meta.keep)].index

In [15]:
# Q size 함수를 해주는 이유?
pd.DataFrame({'count' : meta.groupby(['role', 'level'])['role'].size()}).reset_index()

위에서 만들어준 정리를 통해 유형별로 데이터를 어떻게 다룰 것인가를 쉽게 선택할 수 있음
ex. 연속형 변수의 경우 통계적 방법, 범주형 변수의 경우 시각화를 통한 탐색 등

In [16]:
# Q .index를 해주는 이유?
# 우리는 지금 varname을 index로 만들어 준 상태. level이 interval이면서, keep인 index들을 Interval에 담겠다.
Interval = meta[(meta["level"] == "interval") & (meta["keep"])].index

In [17]:
Interval

In [18]:
# describe를 통해 interval 변수들의 통계량을 확인 
train[Interval].describe()

**reg variables**
- only ps_reg_03 has missing values  
- the range (min to max) differs between the variables. We could apply scaling (e.g. StandardScaler), but it depends on the classifier we will want to use.

**car variables**
- ps_car_12 and ps_car_15 have missing values
- again, the range differs and we could apply scaling.

**calc variables**
- no missing values
- this seems to be some kind of ratio as the maximum is 0.9
- all three _calc variables have very similar distributions

**Overall**, we can see that the range of the interval variables is rather small. Perhaps some transformation (e.g. log) is already applied in order to anonymize the data?

### Ordinal variables

In [28]:
# .index : 데이터 프레임이 아닌 인덱스만 뽑는 것.
Ordinal = meta[(meta["level"] == "ordinal") & (meta["keep"])].index
Ordinal
# train[Ordinal].describe()

In [29]:
Ordinal = meta[(meta["level"] == "ordinal") & (meta["keep"])].index
train[Ordinal].describe()

### Binary variables

In [20]:
Binary = meta[(meta["level"] == 'binary') & (meta["keep"])].index
# describe를 통해 Ordinal 변수들의 통계량을 확인 

train[Binary].describe()

위의 통계량을 바탕으로 Binary 데이터를 살펴보자  
1) 결측치 확인  
결측치 X  

2) 변수들 사이의 범위 확인  
Binary 데이터이기 때문에 범위를 확인할 필요는 없다. (0 or 1)    

3) Target 변수 확인    
Binary에는 Target 변수까지 포함되어 있다.  
그렇기 때문에 Target 변수에 대한 통계량을 확실하게 짚고 가야한다  
Target 데이터의 평균을 살펴보면 이 대회의 핵심을 알 수 있다.  
데이터는 0 or 1 이기 때문에 균형이 맞기 위해선 평균이 0.5가 되어야 한다.  
하지만 Target 데이터의 평균은 0.0364로 보인다. (굉장히 Imbalanced하다. 0이 훨씬 많아보인다)  
대회의 Metric으로 Normalized Gini Coefficient를 사용하는 이유  


**Imbalanced Class 처리**  
보험이라는 도메인 특성상 불균형적인 타겟값은 사실 일반적이다.  
보험을 청구하는 경우 (1) 보다 하지 않는 경우 (0)이 굉장히 많은 것을 위에서 확인하였다.  
Imbalanced한 데이터는 일반적으로 Undersampling 혹은 Oversampling으로 처리한다.  
UnderSampling: 0이 1보다 훨씬 많으므로 0인 데이터를 줄여 균형을 맞춰준다.  
OverSampling: 0이 1보다 훨씬 많으므로 1인 데이터를 늘려 균형을 맞춰준다.  
선택방법: 본인의 결정이지만 보통 데이터셋의 크기를 기준으로 선택한다.  
데이터가 너무 많으면, 오버샘플링 시 너무 많은 Cost가 들어가게 된다(시간, 컴퓨팅파워)  
이 커널은 데이터가 많은편이라고 판단하여 언더샘플링을 수행하였다.  

커널을 떠나서 불균형 방법을 해소하는 방법은 완벽가이드 책에 나왔던 SMOTE란 방법이 있다. (오버샘플링에 해당)  
참고: https://datascienceschool.net/view-notebook/c1a8dad913f74811ae8eef5d3bedc0c3/  
추가로 Class_weights라는 방법도 존재한다. (불균형 class의 가중치를 조정)  
참고: https://scikit-learn.org/stable/modules/generated/sklearn.utils.class_weight.compute_class_weight.html  
성능을 확인하고 어떤 방법으로 불균형을 해소할지 판단하면 된다.  

In [22]:
# 언더샘플링 비율을 지정해주기 위함 
desired_apriori=0.10

# target 변수의 클래스에 따른 인덱스 지정 
idx_0 = train[train["target"] == 0].index
idx_1 = train[train["target"] == 1].index

# 지정해준 인덱스로 클래스의 길이(레코드 수) 지정
nb_0 = len(train.loc[idx_0])
nb_1 = len(train.loc[idx_1])

# 언더샘플링 수행
undersampling_rate = ((1-desired_apriori)*nb_1)/(nb_0*desired_apriori)
undersampled_nb_0 = int(undersampling_rate*nb_0)
print('target=0에 대한 언더샘플링 비율: {}'.format(undersampling_rate))
print('언더샘플링 전 target=0 레코드의 개수: {}'.format(nb_0))
print('언더샘플링 후 target=0 레코드의 개수: {}'.format(undersampled_nb_0))

# 언더샘플링 비율이 적용된 개수 만큼 랜덤하게 샘플을 뽑아서 그 인덱스를 저장
undersampled_idx = shuffle(idx_0, random_state=37, n_samples=undersampled_nb_0)

# 언더샘플링 인덱스와 클래스 1의 인덱스를 리스트로 저장
idx_list = list(undersampled_idx) + list(idx_1)

# Return undersample data frame
df_train = train.loc[idx_list].reset_index(drop=True)

### Data Quality Checks

Checking missing values  
Missings are reprensented as -1

In [36]:
vars_with_missing = []

for f in train.columns:
    missings = train[train[f] == -1][f].count()
    if missings > 0:
        vars_with_missing.append(f)
        missings_perc = missings/train.shape[0]
        
        print('Variable {} has {} records ({:2%}) with missing values'.format(f,missings,missings_perc))
        
print('In total, there are {} variables with missing values'.format(len(vars_with_missing)))
        
        

- ps_car_03_cat and ps_car_05_cat have a large proportion of records with missing values. Remove these variables.
- For the other categorical variables with missing values, we can leave the missing value -1 as such.
- ps_reg_03 (continuous) has missing values for 18% of all records. Replace by the mean.
- ps_car_11 (ordinal) has only 5 records with misisng values. Replace by the mode.
- ps_car_12 (continuous) has only 1 records with missing value. Replace by the mean.
- ps_car_14 (continuous) has missing values for 7% of all records. Replace by the mean.

In [37]:
from sklearn.preprocessing import Imputer

# Dropping the variables with too many missing values
vars_to_drop = ['ps_car_03','ps_car_05']
train.drop(vars_to_drop, inplace = Truem axis=1)  #axis=0이면 열을 중심으로 axis=1이면 행을 중심으로.
meta,loc[(var_to_drop),'keep'] = False    #지운 데이터를 메타 데이터에 반영한다.

# Imputing with the mean or mode
# Imputing: 결측치가 많은 결측자료가 있을 때 표준이나, 대표성이 있는 다른 데이터를 활용하여 대체 될 수 있는 값들로 계산하여 입력 하는 과정을 일컫는다.
# 최빈값(mode): 가장 빈번하게 관찰/측정되는 값

mean_imp = Imputer(missing_values = -1, strategy = 'mean', axis=0)
mode_imp = Imputer(missing_values = -1, strategy = 'most_frequent', axis =0)
#fit_transform 함수는 이차원 배열을 입력으로 받으므로 [[]]를 사용. 그러나 다시 대입할 때는 1차원으로 만들어줘야 되니까 ravel() 함수로 flatten.
train['ps_reg_03'] = mean_imp.fit_transform(train[['ps_reg_03']]).ravel()
train['ps_car_12'] = mean_imp.fit_transform(train[['ps_car_12']]).ravel()
train['ps_car_14'] = mean_imp.fit_transform(train[['ps_car_14']]).ravel()
train['ps_car_11'] = mode_imp.fit_transform(train[['ps_car_11']]).ravel()


https://www.kaggle.com/kongnyooong/porto-seguro-eda-for-korean